In [1]:
import geopandas as gpd
from shapely.geometry import MultiPolygon
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper
from bokeh.palettes import brewer
from bokeh.resources import CDN
from bokeh.embed import file_html
from matplotlib.colors import to_hex
import matplotlib.pyplot as plt

output_notebook()

Loading BokehJS ...

In [2]:
gdf = gpd.read_file('../data/map_data.geojson')
gdf.head(2)

,Province,LocalAuth,UC implem,UC_date_number,geometry
0,England,Hartlepool,Mar-15,1.425168e+18,"MULTIPOLYGON (((-1.27025 54.72717, -1.27251 54..."
1,England,Middlesbrough,Nov-15,1.446336e+18,"MULTIPOLYGON (((-1.23003 54.58411, -1.23432 54..."


In [3]:
def filter_out_small_areas(x, min_area=1e-4):
    return MultiPolygon([i for i in x if i.area > min_area])   

gdf['geometry'] = gdf['geometry'].apply(filter_out_small_areas)  # filtering out small areas
gdf['geometry'] = gdf['geometry'].simplify(0.0015)               # simplifying remaining areas
geojson = gdf.to_json()                                          # converting to geojson

In [4]:
p = figure(
    title='Universal Credit Rollout', 
    plot_height=800, plot_width=800,
    background_fill_color="lightgrey", 
    tooltips=[
        ('Local Authority', '@{LocalAuth}'), 
        ('Implementation Date', '@{UC implem}')
    ],
    tools="pan,wheel_zoom,box_zoom,reset,save",
    match_aspect=True,
    aspect_scale=1,
)

cmap = plt.cm.get_cmap('Blues')
color_mapper = LinearColorMapper(
    palette = [to_hex(cmap(i)) for i in range(256)], 
    low = gdf['UC_date_number'].min(), 
    high = gdf['UC_date_number'].max()
)

p.patches(
    'xs', 'ys', 
    source=GeoJSONDataSource(geojson=geojson), 
    fill_color={'field' :'UC_date_number', 'transform' : color_mapper}, fill_alpha=1,
    line_color='black', line_width=0.15
)

p.toolbar.logo = None # remove Bokeh logo

In [5]:
show(p)

In [6]:
html = file_html(p, CDN, "Universal Credit Map")
with open("./interactive_map.html", "w") as f:
    f.write(html)